In [20]:
import os
import random
import string
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
from pyspark import SparkConf, SparkContext
import pandas as pd
import json

conf = SparkConf().setAppName("appName").setMaster("local")
sc = SparkContext.getOrCreate(conf=conf)
spark = SparkSession(sc)

# Load data https://media.githubusercontent.com/media/RemiDufeu/TPA-MBDS/main/DATA%20extraction/dataSources/Immatriculation.csv
df = spark.read.csv("Immatriculation.csv", header=True, inferSchema=True)

# get all the different marques
marques = df.select("MARQUE").distinct().collect()

# create a json object with the format [
#  {
#   "id": "marque"  # the name of the marque without capital letters and spaces
#   "name": "marque",   # the name of the marque
#   },]
df.show(5)
marques_json = []
for marque in marques:
    marque = marque[0]
    id = marque.lower().replace(" ", "")
    marques_json.append({"id": id, "name": marque})
marques_json = json.dumps(marques_json)

# group by model and count the number of cars and get the average price
df = df.groupBy("MARQUE", "NOM").agg({"PRIX": "avg", "NOM": "count"})

df.show(5)
# create a json object with the format [
#  {
#   "id": "ttvalue"  # tt+random value
#   "marque": "marque",   # the name of the marque
#   "name": "nom",   # the name of the marque
#   "parent": "marque" # the name of the marque without capital letters and spaces
#   "value": "count" Count of cars nom
#   "price": "avg" Average price of cars PRIX
#   },]
df_json = []
for row in df.collect():
    marque = row[0]
    nom = row[1]
    count = row[3]
    avg = row[2]
    id = "tt" + ''.join(random.choice(string.digits) for _ in range(5))
    parent = marque.lower().replace(" ", "")
    df_json.append({"id": id, "marque":marque, "name": nom, "parent": parent, "value": count, "price": avg})    
df_json = json.dumps(df_json)

# Merge the two json objects
marques_json = marques_json[:-1] + "," + df_json[1:]



# write the json object to a file with \n
with open("data.json", "w") as f:
    f.write(marques_json)

+---------------+----------+------------+---------+--------+--------+--------+-------+--------+-----+
|Immatriculation|    Marque|         Nom|Puissance|Longueur|NbPlaces|NbPortes|Couleur|Occasion| Prix|
+---------------+----------+------------+---------+--------+--------+--------+-------+--------+-----+
|     3176 TS 67|   Renault| Laguna 2.0T|      170|  longue|       5|       5|  blanc|   false|27300|
|     3721 QS 49|     Volvo|      S80 T6|      272|      tr|       5|       5|   noir|   false|50500|
|     9099 UV 26|Volkswagen|Golf 2.0 FSI|      150| moyenne|       5|       5|   gris|    true|16029|
|     3563 LA 55|   Peugeot|    1007 1.4|       75|  courte|       5|       5|  blanc|    true| 9625|
|     6963 AX 34|      Audi|      A2 1.4|       75|  courte|       5|       5|   gris|   false|18310|
+---------------+----------+------------+---------+--------+--------+--------+-------+--------+-----+
only showing top 5 rows



+----------+-------------+-----------------+----------+
|    MARQUE|          NOM|        avg(PRIX)|count(NOM)|
+----------+-------------+-----------------+----------+
|       Kia|  Picanto 1.1|           8990.0|     22106|
|  Mercedes|         S500|82521.70324110215|    103761|
|Volkswagen|  Polo 1.2 6V| 11369.1118363481|     97133|
|    Jaguar|X-Type 2.5 V6|33876.56799568282|    189012|
|     Dacia|Logan 1.6 MPI|           7500.0|     13973|
+----------+-------------+-----------------+----------+
only showing top 5 rows



In [13]:
import os
import random
import string
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
from pyspark import SparkConf, SparkContext
import pandas as pd
import json

conf = SparkConf().setAppName("appName").setMaster("local")
sc = SparkContext.getOrCreate(conf=conf)
spark = SparkSession(sc)

# import json file

# Download the file from `url` and save it locally under `file_name`:
import urllib.request
url = "https://raw.githubusercontent.com/RomainCaret/datajson/b9e5a8f537ef7f3375fc1cb48f2b992c35080247/data.json"
urllib.request.urlretrieve(url, "dataOrigin.json")



with open("dataOrigin.json", "r") as f:
    data = json.load(f)

    # transform the data into json format
    data = json.dumps(data)

    # write the json object to a file with \n
    with open("data2.json", "w") as f:
        f.write(data)

    